<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Zelda", "transactions": [{"transaction-id": 5530, "amount": 941}, {"transaction-id": 6480, "amount": 870}, {"transaction-id": 6620, "amount": 905}, {"transaction-id": 6930, "amount": 892}, {"transaction-id": 8023, "amount": 854}, {"transaction-id": 9573, "amount": 871}, {"transaction-id": 9700, "amount": 945}, {"transaction-id": 9948, "amount": 999}]}\n',)

In [5]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [6]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [7]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [8]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Zelda", "transactions": [{"transaction-id": 5530, "amount": 941}, {"transaction-id": 6480, "amount": 870}, {"transaction-id": 6620, "amount": 905}, {"transaction-id": 6930, "amount": 892}, {"transaction-id": 8023, "amount": 854}, {"transaction-id": 9573, "amount": 871}, {"transaction-id": 9700, "amount": 945}, {"transaction-id": 9948, "amount": 999}]}\n',
 '{"id": 1, "name": "Zelda", "transactions": [{"transaction-id": 412, "amount": 743}, {"transaction-id": 1122, "amount": 1151}, {"transaction-id": 1223, "amount": 1307}, {"transaction-id": 1315, "amount": 1044}, {"transaction-id": 1354, "amount": 1062}, {"transaction-id": 2149, "amount": 914}, {"transaction-id": 2451, "amount": 1088}, {"transaction-id": 2567, "amount": 797}, {"transaction-id": 3501, "amount": 924}, {"transaction-id": 4304, "amount": 623}, {"transaction-id": 4504, "amount": 1198}, {"transaction-id": 4656, "amount": 1031}, {"transaction-id": 4739, "amount": 922}, {"transaction-id": 4949, "amount": 1

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [9]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Zelda',
  'transactions': [{'transaction-id': 5530, 'amount': 941},
   {'transaction-id': 6480, 'amount': 870},
   {'transaction-id': 6620, 'amount': 905},
   {'transaction-id': 6930, 'amount': 892},
   {'transaction-id': 8023, 'amount': 854},
   {'transaction-id': 9573, 'amount': 871},
   {'transaction-id': 9700, 'amount': 945},
   {'transaction-id': 9948, 'amount': 999}]},
 {'id': 1,
  'name': 'Zelda',
  'transactions': [{'transaction-id': 412, 'amount': 743},
   {'transaction-id': 1122, 'amount': 1151},
   {'transaction-id': 1223, 'amount': 1307},
   {'transaction-id': 1315, 'amount': 1044},
   {'transaction-id': 1354, 'amount': 1062},
   {'transaction-id': 2149, 'amount': 914},
   {'transaction-id': 2451, 'amount': 1088},
   {'transaction-id': 2567, 'amount': 797},
   {'transaction-id': 3501, 'amount': 924},
   {'transaction-id': 4304, 'amount': 623},
   {'transaction-id': 4504, 'amount': 1198},
   {'transaction-id': 4656, 'amount': 1031},
   {'transaction-id'

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [10]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 39,
  'name': 'Alice',
  'transactions': [{'transaction-id': 953, 'amount': 675},
   {'transaction-id': 1559, 'amount': 652},
   {'transaction-id': 2910, 'amount': 690},
   {'transaction-id': 4389, 'amount': 680},
   {'transaction-id': 4850, 'amount': 663},
   {'transaction-id': 6049, 'amount': 648},
   {'transaction-id': 6115, 'amount': 696},
   {'transaction-id': 6939, 'amount': 651},
   {'transaction-id': 7572, 'amount': 625},
   {'transaction-id': 7581, 'amount': 645},
   {'transaction-id': 8145, 'amount': 690},
   {'transaction-id': 8491, 'amount': 698},
   {'transaction-id': 9053, 'amount': 620},
   {'transaction-id': 9925, 'amount': 570}]},
 {'id': 50,
  'name': 'Alice',
  'transactions': [{'transaction-id': 1049, 'amount': 3032},
   {'transaction-id': 7756, 'amount': 3094},
   {'transaction-id': 9179, 'amount': 3206}]},
 {'id': 70,
  'name': 'Alice',
  'transactions': [{'transaction-id': 54, 'amount': 1189},
   {'transaction-id': 224, 'amount': 1114},
   {'transaction-i

In [11]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 14},
 {'name': 'Alice', 'count': 3},
 {'name': 'Alice', 'count': 52},
 {'name': 'Alice', 'count': 73},
 {'name': 'Alice', 'count': 30})

In [12]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(14, 3, 52, 73, 30)

In [13]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

41.3425

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [14]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 953, 'amount': 675},
  {'transaction-id': 1559, 'amount': 652},
  {'transaction-id': 2910, 'amount': 690},
  {'transaction-id': 4389, 'amount': 680},
  {'transaction-id': 4850, 'amount': 663},
  {'transaction-id': 6049, 'amount': 648},
  {'transaction-id': 6115, 'amount': 696},
  {'transaction-id': 6939, 'amount': 651},
  {'transaction-id': 7572, 'amount': 625},
  {'transaction-id': 7581, 'amount': 645},
  {'transaction-id': 8145, 'amount': 690},
  {'transaction-id': 8491, 'amount': 698},
  {'transaction-id': 9053, 'amount': 620},
  {'transaction-id': 9925, 'amount': 570}],
 [{'transaction-id': 1049, 'amount': 3032},
  {'transaction-id': 7756, 'amount': 3094},
  {'transaction-id': 9179, 'amount': 3206}],
 [{'transaction-id': 54, 'amount': 1189},
  {'transaction-id': 224, 'amount': 1114},
  {'transaction-id': 452, 'amount': 1096},
  {'transaction-id': 1260, 'amount': 1069},
  {'transaction-id': 1294, 'amount': 1182},
  {'transaction-id': 1368, 'amount': 1218},
  {'t

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 953, 'amount': 675},
 {'transaction-id': 1559, 'amount': 652},
 {'transaction-id': 2910, 'amount': 690})

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(675, 652, 690)

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

958.9578520892544

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [18]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [19]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [20]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [21]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [22]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 96), ('Alice', 96), ('Alice', 104), ('Alice', 104), ('Bob', 72), ('Bob', 72), ('Bob', 78), ('Bob', 78), ('Charlie', 107), ('Charlie', 107), ('Charlie', 112), ('Charlie', 113), ('Dan', 127), ('Dan', 127), ('Dan', 137), ('Dan', 137), ('Edith', 108), ('Edith', 108), ('Edith', 234), ('Frank', 161), ('Frank', 163), ('Frank', 176), ('Frank', 180), ('George', 67), ('George', 68), ('George', 72), ('George', 73), ('Hannah', 139), ('Hannah', 140), ('Hannah', 149), ('Hannah', 153), ('Ingrid', 132), ('Ingrid', 132), ('Ingrid', 143), ('Ingrid', 143), ('Jerry', 164), ('Jerry', 165), ('Jerry', 175), ('Jerry', 177), ('Kevin', 144), ('Kevin', 144), ('Kevin', 156), ('Kevin', 156), ('Laura', 104), ('Laura', 104), ('Laura', 192), ('Michael', 107), ('Michael', 108), ('Michael', 116), ('Michael', 117), ('Norbert', 107), ('Norbert', 107), ('Norbert', 115), ('Norbert', 117), ('Oliver', 72), ('Oliver', 72), ('Oliver', 78), ('Oliver', 78), ('Patricia', 120), ('Patricia', 120), ('Patricia', 130), ('Pa

In [23]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 400), ('Bob', 300), ('Charlie', 439), ('Dan', 528), ('Edith', 450), ('Frank', 680), ('George', 280), ('Hannah', 581), ('Ingrid', 550), ('Jerry', 681), ('Kevin', 600), ('Laura', 400), ('Michael', 448), ('Norbert', 446), ('Oliver', 300), ('Patricia', 500), ('Quinn', 398), ('Ray', 582), ('Sarah', 200), ('Tim', 699), ('Ursula', 553), ('Victor', 440), ('Wendy', 450), ('Xavier', 250), ('Yvonne', 520), ('Zelda', 550)]
CPU times: user 172 ms, sys: 10.1 ms, total: 182 ms
Wall time: 652 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [24]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [25]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Zelda,"[{'transaction-id': 5530, 'amount': 941}, {'tr..."
1,1,Zelda,"[{'transaction-id': 412, 'amount': 743}, {'tra..."
2,2,Hannah,"[{'transaction-id': 955, 'amount': 9}, {'trans..."
3,3,Tim,"[{'transaction-id': 186, 'amount': 306}, {'tra..."
4,4,Tim,"[{'transaction-id': 2267, 'amount': -250}, {'t..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [26]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 284 ms, sys: 21.1 ms, total: 305 ms
Wall time: 1.98 s


name
Alice      400
Bob        300
Charlie    439
Dan        528
Edith      450
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [27]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Zelda', 'amount': 941, 'transaction-id': 5530},
 {'id': 0, 'name': 'Zelda', 'amount': 870, 'transaction-id': 6480},
 {'id': 0, 'name': 'Zelda', 'amount': 905, 'transaction-id': 6620})

In [28]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Zelda,941,5530
1,0,Zelda,870,6480
2,0,Zelda,905,6620
3,0,Zelda,892,6930
4,0,Zelda,854,8023


In [29]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 241 ms, sys: 27.1 ms, total: 268 ms
Wall time: 1.6 s


name
Alice       16537
Bob         17483
Charlie     24527
Dan         18827
Edith       20075
Frank       20573
George      13737
Hannah      12224
Ingrid      41486
Jerry       17985
Kevin       14766
Laura       15135
Michael     17139
Norbert     12080
Oliver       9022
Patricia    16026
Quinn       26858
Ray         24392
Sarah       16588
Tim         16016
Ursula      22005
Victor      11868
Wendy       26816
Xavier       7857
Yvonne      22132
Zelda       37846
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [30]:
client.shutdown()